In [2]:
cd ../

/Users/rachelbittner/Dropbox/MARL/repos/motif


In [3]:
import motif
import os
import tempfile as tmp
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

# Make Ground Truth Seeds

In [4]:
import medleydb as mdb
import numpy as np
import mir_eval
import csv

In [12]:
mtracks = mdb.load_melody_multitracks()

In [ ]:
def get_ground_truth_seeds(mtrack):
    sns.set_style("dark")
    print mtrack.track_id
    mtrack.load_melody_annotations()
    data = np.array(mtrack.melody3_annotation)
    mel3 = data[:, 1:]
    times = data[:, 0]
    n_times, n_mels = mel3.shape

    c_index = []
    c_times = []
    c_freqs = []

    seeds = []
    idx = 0
    plt.figure()
    for mel_num in range(n_mels):
        nonzero = mel3[:, mel_num] > 0
        nonzero_diff = np.diff(nonzero.astype(int))

        contour_starts = list(np.where(nonzero_diff == 1)[0] + 1)
        contour_ends = list(np.where(nonzero_diff == -1)[0] + 1)

        if contour_ends[0] < contour_starts[0]:
            contour_starts.insert(0, 0)

        if len(contour_ends) < len(contour_starts):
            contour_ends.append(len(nonzero_diff))

        for s, e in zip(contour_starts, contour_ends):

            t = times[s:e]
            f = mel3[s:e, mel_num]

            contour_diff = np.abs(np.diff(mir_eval.melody.hz2cents(f)))

            splits = list(np.where(contour_diff > 50)[0] + 1)

            if len(splits) > 0:
                if splits[-1] == len(contour_diff):
                    splits = splits[:-1]
                split_starts = [0] + splits
                split_ends = splits + [len(t) - 1]

                for ss, se in zip(split_starts, split_ends):

                    tt = t[ss:se]
                    ii = idx * np.ones(tt.shape)
                    idx += 1
                    ff = f[ss:se]
                    c_index.append(ii)
                    c_times.append(tt)
                    c_freqs.append(ff)
                    seed_idx = int(round(len(tt)/2))
                    seeds.append([int(round(44100*tt[seed_idx])), ff[seed_idx]])
                    plt.plot(tt, ff)
            else:
                i = idx * np.ones(t.shape)
                idx += 1
                c_index.append(i)
                c_times.append(t)
                c_freqs.append(f)
                seed_idx = int(round(len(t)/2))
                seeds.append([int(round(44100*t[seed_idx])), f[seed_idx]])
                plt.plot(t, f)

    plt.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')
    plt.tick_params(axis='y', which='both', bottom='off', top='off', labelleft='off' )
#     plt.show()
    return seeds, c_index, c_times, c_freqs

In [13]:
track_ids = []
for mtrack in mtracks:

#     output_path = '/Users/rachelbittner/Documents/HLL_Data/mdb_seeds_mel3/{}_gt_seeds.csv'.format(mtrack.track_id)
#     if os.path.exists(output_path):
#         continue

    track_ids.append(mtrack.track_id)
#     seeds, _, _, _ = get_ground_truth_seeds(mtrack)
#     plt.savefig('/Users/rachelbittner/Desktop/contour_art/{}.pdf'.format(mtrack.track_id))
#     with open(output_path, 'w') as fhandle:
#         writer = csv.writer(fhandle, delimiter=',')
#         writer.writerows(seeds)


In [ ]:
track_ids

# HLL Parameter Search

In [16]:
import random
import glob

In [6]:
def parametrize_hll():
    etr = motif.contour_extractors.hll.HLL()
    etr.n_harmonics = random.choice([1, 2, 3, 4, 5, 6])
    etr.f_cutoff = 30
    etr.tracking_gain = random.choice([8e-5, 1e-4, 3e-4, 5e-4, 8e-4, 1e-3, 2e-3, 3e-3, 5e-3])
    etr.min_contour_len_samples = random.choice(
        [0, 10, 441, 2205, 4410, 11025, 22050, 44100, 88200, 132300, 176400, 220500, 441000]
    )
    etr.amplitude_threshold = np.random.uniform(0, 1)
    etr.tracking_update_threshold = np.random.uniform(0, 500)
    
    return etr

In [7]:
def load_melody3_multif0(fpath):
    times = []
    freqs = []
    with open(fpath, 'r') as fhandle:
        reader = csv.reader(fhandle, delimiter=',')
        for line in reader:
            times.append(float(line[0]))
            f = np.array(line[1:], dtype=float)
            f = f[f > 0]
            freqs.append(f)
    return np.array(times), freqs

In [8]:
def run_hll(etr, audio_filepath, seed_fpath):
    tmp_audio = etr._preprocess_audio(
        audio_filepath, normalize_format=True, normalize_volume=True
    )

    contours_fpath = tmp.mktemp('.csv')

    args = [
        "hll",
        "{}".format(tmp_audio),
        "{}".format(seed_fpath),
        "{}".format(contours_fpath),
        "{}".format(etr.n_harmonics),
        "{}".format(etr.f_cutoff),
        "{}".format(etr.tracking_gain),
        "{}".format(etr.min_contour_len_samples),
        "{}".format(etr.amplitude_threshold),
        "{}".format(etr.tracking_update_threshold)
    ]

    os.system(' '.join(args))

    if not os.path.exists(contours_fpath):
        raise IOError(
            "Unable to find HLL output file {}".format(contours_fpath)
        )

    c_numbers, c_times, c_freqs, c_sal = etr._load_contours(contours_fpath)

    os.remove(contours_fpath)
    os.remove(tmp_audio)

    return motif.core.Contours(
        c_numbers, c_times, c_freqs, c_sal, etr.sample_rate, audio_filepath
    )

In [17]:
SEED_PATH = '/Users/rachelbittner/Documents/HLL_Data/mdb_seeds_mel3/'
AUDIO_PATH = '/Datasets/MedleyDB_Mixes/'
ANNOT_PATH = '/Users/rachelbittner/Dropbox/MARL/repos/medleydb/Annotations/'
# file_triples = [
#     ('/Datasets/MedleyDB_Mixes/MusicDelta_80sRock_MIX.wav',
#      '/Users/rachelbittner/Documents/HLL_Data/mdb_seeds_mel3/MusicDelta_80sRock_gt_seeds.csv',
#      '/Users/rachelbittner/Dropbox/MARL/repos/medleydb/Annotations/MusicDelta_80sRock_ANNOTATIONS/MusicDelta_80sRock_MELODY3.csv'
#     )
# ]

grid_search_trackids = np.random.choice(track_ids, 10)
file_triples = []
for tid in grid_search_trackids:
    audio_file = os.path.join(AUDIO_PATH, '{}_MIX.wav'.format(tid))
    seed_file = os.path.join(SEED_PATH, '{}_gt_seeds.csv'.format(tid))
    annot_file = os.path.join(ANNOT_PATH, '{}_ANNOTATIONS'.format(tid), '{}_MELODY3.csv'.format(tid))
    file_triples.append((audio_file, seed_file, annot_file))

In [ ]:
grid_scores = []
n_iter = 5
for i in range(n_iter):
    print("running iteration {}".format(i))

    etr = parametrize_hll()
    this_grid_score = {}
    this_grid_score['params'] = {
        'n_harmonics': etr.n_harmonics,
        'f_cutoff': etr.f_cutoff,
        'tracking_gain': etr.tracking_gain,
        'min_contour_len_samples': etr.min_contour_len_samples,
        'amplitude_threshold': etr.amplitude_threshold,
        'tracking_update_threshold': etr.tracking_update_threshold
    }

    recall = []
    precision = []
    accuracy = []
    for audio_file, seed_file, annot_file in file_triples[:1]:
        print("    {}".format(os.path.basename(audio_file)))

        ctr = run_hll(etr, audio_file, seed_file)
        est_time, est_freqs = ctr.to_multif0_format()
        ref_time, ref_freqs = load_melody3_multif0(annot_file)

        scores = mir_eval.multipitch.evaluate(np.array(ref_time), ref_freqs, np.array(est_time), est_freqs)
        recall.append(scores['Recall'])
        precision.append(scores['Precision'])
        accuracy.append(scores['Accuracy'])

    this_grid_score['recall'] = np.array(recall)
    this_grid_score['precision'] = np.array(precision)
    this_grid_score['accuracy'] = np.array(accuracy)
    grid_scores.append(this_grid_score)

In [11]:
grid_scores

{0: {'accuracy': array([ 0.08835979]),
  'params': {'amplitude_threshold': 0.11831388360162332,
   'f_cutoff': 30,
   'min_contour_len_samples': 88200,
   'n_harmonics': 4,
   'tracking_gain': 0.002,
   'tracking_update_threshold': 103.0641405893779},
  'precision': array([ 0.08893228]),
  'recall': array([ 0.93209302])},
 1: {'accuracy': array([ 0.]),
  'params': {'amplitude_threshold': 0.9399637981238619,
   'f_cutoff': 30,
   'min_contour_len_samples': 0,
   'n_harmonics': 2,
   'tracking_gain': 0.0001,
   'tracking_update_threshold': 129.91276045650818},
  'precision': array([ 0.]),
  'recall': array([ 0.])},
 2: {'accuracy': array([ 0.24496778]),
  'params': {'amplitude_threshold': 0.3438473278239499,
   'f_cutoff': 30,
   'min_contour_len_samples': 22050,
   'n_harmonics': 1,
   'tracking_gain': 8e-05,
   'tracking_update_threshold': 40.11279095864301},
  'precision': array([ 0.27132534]),
  'recall': array([ 0.71604651])},
 3: {'accuracy': array([ 0.02125088]),
  'params': {'amp

In [15]:
np.random.choice?